<center>
<img src="https://2223moodle.isel.pt/pluginfile.php/1/theme_adaptable/logo/1631635830/logo-isel_inv3.png" width="250">
<h3>Licenciatura em Engenharia Informática e Multimédia</h3><br>
<br>
<h2>Processamento de Imagem e Visão (PIV)</h3>
<h3>2º Trabalho Laboratorial – Estimação e classificação de movimento </h3>
<br>

<br>
Trabalho Realizado por:<br>
Gonçalo Silva <b>A48328</b><br>
Diogo Lobo <b>A48168</b><br>
Turma 52D<br><br>
Docente: João Pedro Costa <br>
<br>
27 de Novembro 2024
</center>


***

<br>
<a id='section0'></a>
<center><h2><b>Índice</b></2></center>

- [Introdução](#section1)
- [Desenvolvimento](#section2)
  - [Deteção de Movimento](#section3)
  - [Deteção de Contornos](#section4)
  - [Deteção da Mão](#section5)
  - [Implementação da Interação](#section6)
- [Conclusão](#section7)
- [Bibliografia](#section8)

***

<a id='section1'></a>
## <center><b>Introdução</b></center>
<div style="text-align: justify">
<h3> Enquadramento</h3>

No contexto da Unidade Curricular de Processamento de Imagem e Visão, foi proposto como segundo trabalho o desenvolvimento de um algoritmo de estimação e classificação de movimento para integração numa aplicação gráfica interativa;<br>

<h3> Objetivo</h3>

Pretende-se desenvolver um algoritmo que, ao utilizar a camara do computador, seja possivel mover um objeto com gestos da mão humana utilizando a interface Pygame.

<h3> Métodos</h3>

Para o desenvolvimento deste segundo trabalho é necessário utilizar python e várias bibliotecas como opencv, numpy, time e pygame. É tambem necessário uma câmara para que se possa realizar a interação entre o algoritmo e o ser humano.<br>

</div>

In [ ]:
# Imports
import cv2
import numpy as np
import time
import pygame
from pygame.locals import QUIT
from pygame import surfarray


pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


***
<a id='section2'></a>
# <center><b>Desenvolvimento</b></center>
<div style="text-align: justify">

Valores utilizados para o funcionamento do trabalho.

In [ ]:
#Inicialização dos tamanhos da janela e objetos
window_width = 1280
window_height = 700
cube_size = 50
cube_x = window_width // 4 - cube_size
cube_y = window_height // 2 - cube_size
cube_color = (0, 0, 255)
previousContour = None
radius = 20
cyNew = None
cxNew = None
cxOld = None
cyOld = None
oldArea = None
newArea = None

<a id='section3'></a>
# <center><b>Deteção de Movimento</b></center>
<div style="text-align: justify">

O primeiro passo realizado no nosso trabalho foi a familiarização com o pygame, e deteção de movimento em geral. Assim, com os exercícios práticos realizados em aula fizemos um simples teste que permite detectar qualquer movimento e mostrar os resultados numa janela de pygame.

Nesta porção de código utilizando a camara do computador é verificada a cada frame se houve uma mudança a comparar com a frame anterior e assinala com um quadrado vermelho, caso exista uma diferença, o local dessa diferença.

Para que isto seja possivel é utilizado o threshold da frame para destacar áreas com mudanças significativas e analisando os componentes assinalados com a função *connectedComponentsWithStats*. Com o que se obtem desta função é possivel comparar as duas frames para alteração entre as frames.

In [ ]:
#Start da camara
camera = 0
cam = cv2.VideoCapture(camera)

running = True


pygame.init()

window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Webcam in Pygame")

running = True
fonte = pygame.font.SysFont("Arial", 32)
previousFrame = None
thr = 10
kernelSize = 9
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernelSize,kernelSize),(-1,-1))

if cam.isOpened():
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        flag, frame = cam.read()
        if flag:

            frameE = cv2.medianBlur(frame,kernelSize)

            if previousFrame is None:
                previousFrame = frame

            diff_image = cv2.absdiff(frame,previousFrame)
            _ ,bw1 = cv2.threshold(diff_image,thr,255,cv2.THRESH_BINARY)
            _, bw2 = cv2.threshold(np.uint8(bw1.sum(axis=2)),1,255,cv2.THRESH_BINARY)

            erode_image = cv2.erode(bw2,kernel)
            dilate_image = cv2.dilate(erode_image,kernel,iterations=2)

            (num_labels,label_image,stats,centroids) = cv2.connectedComponentsWithStats(dilate_image)

            previousFrame = frame.copy()

            motionFlag = False

            for i in range(1,num_labels):
                if(stats[i,4]>0.05*bw2.shape[0]*bw2.shape[1]):
                    cv2.rectangle(frame,(stats[i,0],stats[i,1]),(stats[i,0]+stats[i,2],stats[i,1]+stats[i,3]),(0,0,255),thickness=2)
                    motionFlag = True

            if motionFlag:
                texto_surface = fonte.render("Motion Detected", True, (255, 0, 0))  # Texto em vermelho
                window.blit(texto_surface, (10, 10))  # Posição do texto

            ##### Vizualização da câmara no Pygame e sua localização ######

            # Converte de BGR (OpenCV) para RGB (Pygame)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = np.rot90(frame)  # Pygame espera a imagem transposta
            pygame_surface = pygame.surfarray.make_surface(frame)
            window.blit(pygame_surface, (window_width // 2, window_height // 7))  # Centraliza no ecrã

        pygame.display.update()  # Atualiza o ecrã

    cam.release()
    pygame.quit()

else:
    print("Error: Could not open Camera")


<a id='section4'></a>
# <center><b>Deteção de Contornos</b></center>
<div style="text-align: justify">

Agora com uma ideia acerca de como detectar movimentos, inciamos o próximo passo, ***deteção da mão***. Para isto começamos por explorar o método GoodFeaturesToTrack.

O método goodFeaturesToTrack() é utilizado para detetar pontos de interesse em imagens, como arestas e cantos, que são estáveis e fáceis de seguir em sequências de vídeo.

No entanto não é suficiente para detetar a mão, assim, para garantir que a deteção se concentra exclusivamente na mão, realizamos uma segmentação baseada na cor da pele (opção b do H.3). Esse processo de segmentação permite ajudar na isolação da mão do restante da imagem.


In [ ]:
import cv2
import numpy as np
import pygame

# Iniciar a câmara e Pygame
pygame.init()
cap = cv2.VideoCapture(0)
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Hand Detection with GoodFeaturesToTrack1")

running = True

while running:
    ret, frame = cap.read()
    if not ret:
        break

    # Converte a imagem para escala de cinzento
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Segmentação baseada na cor da pele (opcional)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Aplica a máscara e detecta as características
    masked_gray = cv2.bitwise_and(gray, gray, mask=mask)

    # Detecta características usando goodFeaturesToTrack
    corners = cv2.goodFeaturesToTrack(masked_gray, maxCorners=50, qualityLevel=0.01, minDistance=10)
    if corners is not None:
        for corner in corners:
            x, y = corner.ravel()
            cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)  # Desenha círculos nos pontos detectados

    # Exibe a imagem com os pontos detectados no Pygame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_rgb = np.rot90(frame_rgb)
    pygame_surface = pygame.surfarray.make_surface(frame_rgb)
    window.blit(pygame_surface, (0, 0))
    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

cap.release()
pygame.quit()


<a id='section5'></a>
# <center><b>Deteção da Mão</b></center>
<div style="text-align: justify">

Com os resultados obtidos anteriormente, observou-se que múltiplos contornos eram detetados na imagem, o que dificultava a precisão do algoritmo. Contudo, para alcançar o objetivo de detetar exclusivamente a mão, adotou-se a estratégia de considerar apenas o **maior contorno** detetado em cada frame.

A suposição subjacente é que a mão será o maior objeto em movimento no campo de visão, devido ao seu destaque durante a interação com o sistema.
Este procedimento aumenta a precisão do sistema ao eliminar detecções irrelevantes, como pequenos objetos ou ruídos que possam estar presentes no ambiente.

Por se utilizar a cor para a segmentação é capaz de existir locais onde não funcione muito bem pelo fundo ter uma cor existente no intervalo ou ao utilizar uma luz amarela tambem piora os resultados obtidos no trabalho.

In [ ]:
import cv2
import numpy as np
import pygame

# Inicia a câmara e Pygame
pygame.init()
cap = cv2.VideoCapture(0)
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Hand Detection with GoodFeaturesToTrack")
fonte = pygame.font.SysFont("Arial", 64)

running = True

while running:
    ret, frame = cap.read()
    if not ret:
        break

    # Converte a imagem para o espaço HSV e escala de cinzento
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Segmentação baseada na cor da pele
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)


    mask = cv2.erode(mask, np.ones((3, 3), np.uint8), iterations=2)
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=2)

    # Encontra contornos da máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Inicializa flag para verificar se a mão foi detectada
    hand_detected = False

    if contours:
        # Seleciona o maior contorno, assumindo que é a mão
        largest_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(largest_contour) > 5000:  # Filtra áreas pequenas (ajustar conforme necessário)
            hand_detected = True

            # Desenha o contorno da mão
            cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 2)

            # Cria uma máscara apenas para a mão detetada
            hand_mask = np.zeros_like(mask)
            cv2.drawContours(hand_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

            # Aplica a máscara da mão à imagem em escala de cinzento
            hand_region = cv2.bitwise_and(gray, gray, mask=hand_mask)

            # Detecta características usando goodFeaturesToTrack apenas na região da mão
            corners = cv2.goodFeaturesToTrack(hand_region, maxCorners=50, qualityLevel=0.01, minDistance=10)
            if corners is not None:
                for corner in corners:
                    x, y = corner.ravel()
                    cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)  # Desenha círculos nos pontos detectados

    ##### Vizualização da câmara no Pygame e a sua localização #####

    # Converte de BGR (OpenCV) para RGB (Pygame)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.rot90(frame)  # Pygame espera a imagem transposta
    pygame_surface = pygame.surfarray.make_surface(frame)
    window.fill((0, 0, 0))  # Limpa a tela antes de atualizar o frame
    window.blit(pygame_surface, (window_width // 2, window_height // 7))  # Centraliza na tela

    # Mostra o texto apenas se a mão for detetada
    if hand_detected:
        texto_surface = fonte.render("Hand Detected", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 520, 10))  # Posição do texto

    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

cap.release()
pygame.quit()


<a id='section6'></a>
# <center><b>Implementação da Interação</b></center>
<div style="text-align: justify">


Nesta fase, implementamos a interação da interface gráfica com base nos movimentos detectados da mão. A detecção de movimentos é realizada comparando a posição atual da mão com sua posição anterior (baseada no centroide do maior contorno detectado) e a área do contorno para operações de zoom.

Movimentos Implementados:

- Up (Para Cima):
O movimento "Up" é realizado quando a posição vertical do centroide atual da mão (cyNew) é menor do que a posição vertical anterior (cyOld), com uma margem de pelo menos 10 pixels. Isto, para garantir que pequenos ruídos na detecção não sejam interpretados como movimento.

- Down (Para Baixo):
O movimento "Down" é acionado quando cyNew é maior que cyOld em pelo menos 10 pixels.

- Left (Esquerda):
O movimento "Left" ocorre quando a posição horizontal do centroide (cxNew) é menor que cxOld, com uma diferença de 10 pixels

- Right (Direita):
O movimento "Right" é detectado quando cxNew é maior que cxOld em pelo menos 10 pixels.

Zoom:

- Zoom In:
O "Zoom In" é ativado quando a área do maior contorno atual (newArea) aumenta em pelo menos 2000 pixels em relação à área anterior (oldArea). Esse aumento indica que a mão está se aproximando da câmera.

- Zoom Out:
O "Zoom Out" ocorre quando newArea diminui em pelo menos 2000 pixels, indicando que a mão está se afastando da câmera.


Restrições de Movimento:

- Para evitar que o objeto saia da área visível da interface, as funções canMove() verificam se o próximo movimento está dentro dos limites permitidos, através do raio do objeto e a posição atual.


In [ ]:
def canMove(x,y,r,direction):
    if(direction == 0):
        return (y >= r)                 and ((y<= 250-r) or (x-r>=250) or (y>250+r))   and ((y<= 250-r) or (x+r<388) or (y>250+r))
    if(direction == 1):
        return (y <= window_height - r) and ((y<= 200-r) or (x-r>=250) or (y>200+r))   and ((y<= 200-r) or (x+r<388) or (y>200+r))
    if(direction == 2):
        return (x >= r)   and ((x - r >= 250) or not (200 - r <= y <= 250 + r))
    if(direction == 3):
        return (x <= window_width/2 - r) and ((x + r <= 388) or not (200 - r <= y <= 250 + r))

def convImage(frame):
    # Converte a imagem para o espaço HSV e escala de cinza
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Segmentação baseada na cor da pele
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)


    mask = cv2.erode(mask, np.ones((3, 3), np.uint8), iterations=2)
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=2)

    # Encontra contornos da máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return gray,mask,contours


def moments(largest_contour):
        Mnew = cv2.moments(largest_contour)
        if Mnew['m00'] != 0:
            cxNew = int(Mnew['m10'] / Mnew['m00'])
            cyNew = int(Mnew['m01'] / Mnew['m00'])
            newArea = cv2.contourArea(largest_contour)
        else:
            cxNew, cyNew, newArea = 0, 0, 0  # Fallback values

        return cxNew,cyNew,newArea

In [ ]:
# Inicia a câmara e Pygame
pygame.init()
cap = cv2.VideoCapture(0)
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Hand Detection with GoodFeaturesToTrack")
fonte = pygame.font.SysFont("Arial", 64)
fonte2 = pygame.font.SysFont("Arial", 50)
running = True
player_pos = pygame.Vector2(window.get_width() // 4, window.get_height() // 2)
dt = 0
time.sleep(5)
while running:
    window.fill((0, 0, 0))

    ret, frame = cap.read()
    if not ret:
        break

    gray, mask, contours = convImage(frame)

        # Inicializa flag para verificar se a mão foi detectada
    hand_detected = False

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(largest_contour) > 5000:
            if previousContour is None:
                previousContour = largest_contour.copy()
            hand_detected = True

            cxNew,cyNew,newArea = moments(largest_contour)

            cxOld, cyOld, oldArea = moments(previousContour)

            # Desenha o contorno da mão
            cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 2)

                # Cria uma máscara apenas para a mão detectada
            hand_mask = np.zeros_like(mask)
            cv2.drawContours(hand_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

                # Aplica a máscara da mão à imagem em escala de cinza
            hand_region = cv2.bitwise_and(gray, gray, mask=hand_mask)

                # Detecta características usando goodFeaturesToTrack apenas na região da mão
            corners = cv2.goodFeaturesToTrack(hand_region, maxCorners=50, qualityLevel=0.01, minDistance=10)
            if corners is not None:
                for corner in corners:
                    x, y = corner.ravel()
                    cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)  # Desenha círculos nos pontos detectados

        ##### Vizualização da câmara no Pygame e sua colocação #####

        # Converte de BGR (OpenCV) para RGB (Pygame)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.rot90(frame)  # Pygame espera a imagem transposta
    pygame_surface = pygame.surfarray.make_surface(frame)
    window.fill((0, 0, 0))  # Limpa a tela antes de atualizar o frame
    window.blit(pygame_surface, (window_width // 2, window_height // 7))  # Centraliza na tela
    pygame.draw.rect(window,(255,255,255),(0,200,250,50))
    pygame.draw.rect(window,(255,255,255),(388,200,250,50))
        # Mostra o texto apenas se a mão for detectada
    if hand_detected:
        texto_surface = fonte.render("Hand Detected", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 520, 10))  # Posição do texto
    pygame.draw.circle(window, "red", player_pos, radius)
    keys = pygame.key.get_pressed()
    if keys[pygame.K_w] or (cyNew is not None and cyOld is not None and cyNew < cyOld + 5 and canMove(player_pos.x,player_pos.y, radius, 0)):
        pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
        player_pos.y -= 1000 * dt
        texto_surface = fonte2.render("Up", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto
    if keys[pygame.K_s] or (cyNew is not None and cyOld is not None and cyNew + 5 > cyOld and canMove(player_pos.x,player_pos.y, radius, 1)):
        pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
        player_pos.y += 1000 * dt
        texto_surface = fonte2.render("Down", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto
    if keys[pygame.K_a] or (cxNew is not None and cxOld is not None and cxNew + 5 > cxOld and canMove(player_pos.x,player_pos.y, radius, 2)):
        pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
        player_pos.x -= 1000 * dt
        texto_surface = fonte2.render("Left", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto
    if keys[pygame.K_d] or (cxNew is not None and cxOld is not None and cxNew < cxOld + 5 and canMove(player_pos.x,player_pos.y, radius, 3)):
        pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
        player_pos.x += 1000 * dt
        texto_surface = fonte2.render("Right", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto
    if keys[pygame.K_PLUS] or (newArea is not None and oldArea is not None and newArea + 750 > oldArea):
        pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
        if(radius<1000):
            radius += 100 * dt
        texto_surface = fonte2.render("Zoom In", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto

    if keys[pygame.K_MINUS] or (newArea is not None and oldArea is not None and newArea < oldArea + 750):
        pygame.draw.rect(window, (0, 0, 0), (window_width - 250, window_height - 90, 200, 100))
        if(radius>10):
            radius -= 100 * dt
        texto_surface = fonte2.render("Zoom Out", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 250, window_height - 90))  # Posição do texto


    texto_surface = fonte.render("Movement:", True, (255, 0, 0))  # Texto em vermelho
    window.blit(texto_surface, (window_width - 600, window_height - 100))  # Posição do texto
    previousContour = largest_contour.copy()


    pygame.display.update()
    dt = clock.tick(60) / 1000
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

cap.release()
pygame.quit()

___

<a id='section7'></a>
# <center><b>Conclusão</b></center>
<div style="text-align: justify">

Em conclusão, este segundo trabalho prático permitiu-nos aplicar tanto os conceitos lecionados em aula como os exercicios práticos sobre estimação e classificação de movimento.

Foi possivel utilizar o conhecimento do primeiro trabalho prático, mais precisamente sobre os contornos em 'imagens', neste caso as frames e aprofundar o conhecimento sobre o pygame que não foi muito utilizado neste curso.

Existiram algumas dificuldades na realização do trabalho como no teste do algoritmo por funcionar melhor em certos locais comparando com outros o que dificultou ao testar o trabalho.

Passadas estas dificuldades sentimos que realizámos um bom trabalho funcional onde é possivel movimentar uma bola utilizando a mão.

___

<a id='section8'></a>
# <center><b>Bibliografia</b></center>
<div style="text-align: justify">

[Slides da unidade curricular disponibilizados pelos docentes;](https://2425moodle.isel.pt/course/view.php?id=8502)\
[Documentação OpenCV;](https://docs.opencv.org/4.x/index.html)\
[Documentação pygame;](https://www.pygame.org/docs/)
